**Libraries**

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize


**Variables**

In [3]:
cryptos = ['BTC-USD', 'ATOM-USD', 'USDT-USD', 'DOGE-USD']
start_date = '2022-11-01'
end_date = '2023-11-01'
initial_cap = 1000

**fetch data from yahoo finance**

In [4]:
crypto_data = pd.DataFrame()

for crypto in cryptos:
    ticker = yf.Ticker(crypto)
    data = ticker.history(start=start_date, end=end_date, interval='1d')
    crypto_data[crypto] = data['Close']

crypto_data

,BTC-USD,ATOM-USD,USDT-USD,DOGE-USD
Date,,,,
2022-11-01 00:00:00+00:00,20485.273438,14.105830,0.999924,0.142575
2022-11-02 00:00:00+00:00,20159.503906,13.294817,0.999996,0.127830
2022-11-03 00:00:00+00:00,20209.988281,13.508750,1.000007,0.122592
2022-11-04 00:00:00+00:00,21147.230469,15.038064,1.000076,0.126280
2022-11-05 00:00:00+00:00,21282.691406,15.397510,1.000119,0.124388
...,...,...,...,...
2023-10-27 00:00:00+00:00,33909.800781,7.030354,1.000257,0.067881
2023-10-28 00:00:00+00:00,34089.574219,7.184305,1.000253,0.069001
2023-10-29 00:00:00+00:00,34538.480469,7.316870,1.000387,0.069359


**Metrics**

In [5]:
## Source1 : https://www.kaggle.com/code/jaakkokivisto/pandas-tutorial-for-finance-bigtech-sharpe-ratios
## Source2 : https://www.codearmo.com/blog/sharpe-sortino-and-calmar-ratios-python

def Sharpe_Ratio(balance):
    returns = balance.pct_change().dropna()
    risk_free = (1.02**(1/360))-1 
    mean = returns.mean()
    vol = returns.std()
    sharp_ratio = (mean - risk_free) / vol
    return sharp_ratio


def Sortino_Ratio(balance):
    returns = balance.pct_change().dropna()
    risk_free = (1.02**(1/360))-1 
    mean = returns.mean()
    vol = returns[returns < 0].std()
    sortino_ratio = (mean - risk_free) / vol
    return sortino_ratio

def Net_Profit(balance):
    net_profit = balance.iloc[-1] - initial_cap
    return net_profit

In [15]:
weights = [0.3 , 0.25 , 0.4 , 0.05]

balance = initial_cap
returns = crypto_data.pct_change().dropna()
cumulative_returns = (1 + returns).cumprod()
weighted_culmulative_returns = (cumulative_returns * weights)
weighted_culmulative_returns.sum(axis=1)
weighted_culmulative_returns
# balance*= weighted_culmulative_returns.sum(axis=1)
# balance

,BTC-USD,ATOM-USD,USDT-USD,DOGE-USD
Date,,,,
2022-11-02 00:00:00+00:00,0.295229,0.235626,0.400029,0.044829
2022-11-03 00:00:00+00:00,0.295969,0.239418,0.400033,0.042992
2022-11-04 00:00:00+00:00,0.309694,0.266522,0.400061,0.044285
2022-11-05 00:00:00+00:00,0.311678,0.272893,0.400078,0.043622
2022-11-06 00:00:00+00:00,0.306461,0.256637,0.400066,0.040191
...,...,...,...,...
2023-10-27 00:00:00+00:00,0.496598,0.124600,0.400133,0.023805
2023-10-28 00:00:00+00:00,0.499230,0.127329,0.400132,0.024198
2023-10-29 00:00:00+00:00,0.505805,0.129678,0.400185,0.024324


**Buy & Hold Strategy**

In [7]:
## Source1 : https://www.youtube.com/watch?v=00ejyn7eiw0
## Source2 : https://stackoverflow.com/questions/35365545/calculating-cumulative-returns-with-pandas-dataframe


def buy_and_hold(crypto_data,crypto_weights):
    balance = initial_cap
    returns = crypto_data.pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    weighted_culmulative_returns = (cumulative_returns * crypto_weights).sum(axis=1)
    balance *= weighted_culmulative_returns
    return balance

In [8]:
weights = [0.3 , 0.25 , 0.4 , 0.05]
balance = buy_and_hold(crypto_data,weights)
print(Sharpe_Ratio(balance))
print(Sortino_Ratio(balance))
print(Net_Profit(balance))

0.020886936896078657
0.027773060965495713
72.42321168996364


**Strategy with finding best weights for each metric**

In [9]:
## variables
initial_weights = [0.3 , 0.25 , 0.4 , 0.05]
balance = 1000
returns = crypto_data.pct_change().dropna()
cumulative_returns = (1 + returns).cumprod()

In [10]:
## objective methods

def objective_sharpe_ratio(weights, cumulative_returns, balance):
    weighted_cumulative_returns = (cumulative_returns * weights).sum(axis=1)
    balance *= weighted_cumulative_returns
    returns = balance.pct_change().dropna()
    risk_free = (1.02**(1/365)) - 1 
    mean = returns.mean()
    vol = returns.std()
    sharpe_ratio = (mean - risk_free) / vol
    return -sharpe_ratio

def objective_sortino_ratio(weights, cumulative_returns, balance):
    weighted_cumulative_returns = (cumulative_returns * weights).sum(axis=1)
    balance *= weighted_cumulative_returns
    returns = balance.pct_change().dropna()
    risk_free = (1.02**(1/365)) - 1 
    mean = returns.mean()
    vol = returns[returns < 0].std()
    sortino_ratio = (mean - risk_free) / vol
    return -sortino_ratio


def objective_net_profit(weights,culmulative_returns,balance):
    weighted_cumulative_returns = (culmulative_returns * weights).sum(axis=1)
    balance *= weighted_cumulative_returns
    net_profit = balance.iloc[-1] - 1000
    return -1 * net_profit

In [11]:
## Source1 : https://www.youtube.com/watch?v=iSnTtV6b0Gw
## Source 2 : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
## Source 3 : https://www.youtube.com/watch?v=ZmXimqFmXAQ


### our method for finding the best weights to optimize each metrics

constraints = ({'type': 'eq', 'fun': lambda weights: sum(weights) - 1})
bounds = [(0, 1)] * len(initial_weights)

result_sharpe = minimize(objective_sharpe_ratio, initial_weights, args=(cumulative_returns, balance),
                          method='SLSQP', bounds=bounds, constraints=constraints)

result_sortino = minimize(objective_sortino_ratio, initial_weights, args=(cumulative_returns, balance),
                          method='SLSQP', bounds=bounds, constraints=constraints)

net_profit = minimize(objective_net_profit, initial_weights, args=(cumulative_returns, balance),
                          method='SLSQP', bounds=bounds, constraints=constraints)



## optimal weights
print("Optimal Weights for Maximizing Sharpe Ratio:", result_sharpe.x)
print("Optimal Weights for Maximizing Sortino Ratio:", result_sortino.x)
print("Optimal Weights for Maximizing Net Profit:", net_profit.x)



print('------------------------------------------------------')
### optimal value for each metric

weighted_returns_sharpe = (returns * result_sharpe.x).sum(axis=1)
weighted_returns_sortino = (returns * result_sortino.x).sum(axis=1)
weighted_returns_net_profit = (returns * net_profit.x).sum(axis=1)

cumulative_returns_sharpe = (1 + weighted_returns_sharpe).cumprod()
cumulative_returns_sortino = (1 + weighted_returns_sortino).cumprod()
cumulative_returns_net_profit = (1 + weighted_returns_net_profit).cumprod()

sharpe_ratio = objective_sharpe_ratio(result_sharpe.x, cumulative_returns, balance)
sortino_ratio = objective_sortino_ratio(result_sortino.x, cumulative_returns, balance)
net_profit_metric = objective_net_profit(net_profit.x, cumulative_returns, balance)

print("Best Sharpe Ratio:", -result_sharpe.fun)
print("Best Sortino Ratio:", -result_sortino.fun)  
print("Best Net Profit:", -net_profit.fun)


Optimal Weights for Maximizing Sharpe Ratio: [1.00000000e+00 1.24900090e-16 1.11022302e-16 0.00000000e+00]
Optimal Weights for Maximizing Sortino Ratio: [1.0000000e+00 3.1918912e-16 0.0000000e+00 0.0000000e+00]
Optimal Weights for Maximizing Net Profit: [1.0000000e+00 0.0000000e+00 6.4256556e-09 0.0000000e+00]
------------------------------------------------------
Best Sharpe Ratio: 0.07064732648268994
Best Sortino Ratio: 0.10087820755387344
Best Net Profit: 692.3270020023515
